###  Pasos a seguir en el proceso de 'scraping':

1. Encuentra la URL que quieres 'escrapear'.
2. Inspecciona la página (código fuente).
3. Localiza los datos que necesitas obtener.
4. Desarrolla tu código en Python.
    1. Crea tu sopa
    2. Busca los elementos que cotienen los datos y extráelos
5. Ejecuta tu código y obten los datos.
6. Almacena los datos en el formato requerido.

In [3]:
# Importamos librerías
import requests
from bs4 import BeautifulSoup
import pandas as pd

## Caso 1: Scraping de un catálogo: Labirratorium

In [4]:
URL = "https://www.labirratorium.com/es/67-cervezas-por-estilo?page="
response = requests.get(URL)
html = response.content

# Parsear el HTML con BeautifulSoup
contenido = BeautifulSoup(html, 'lxml')

Queremos obtener un dataFrame con todas las cervezas del catálogo y sus características descritas. Analizamos la página para ver qué tenemos que hacer para conseguirlo

In [5]:
# La web tiene XX páginas con XX cervezas listadas en cada página.
lista_paginas=[]
for i in range(1,50):
    lista_paginas.append(URL+str(i))
    
#Lista con todos los productos que existen la página
productos=[]

for pagina in lista_paginas:
    enlaces = contenido.find_all("a", class_="thumbnail product-thumbnail")
    for enlace in enlaces:
        primer_enlace = enlace['href']
        productos.append(primer_enlace)

In [84]:
dic_productos={"Nombre":[],"Descripción":[]}
for producto in productos:
    response = requests.get(producto)
    soup = BeautifulSoup(response.content, 'html.parser')
    nombre = soup.find_all('h1', {'class': 'product-detail-name'})
    descripcion = soup.find_all('div', {'class': 'product-description'})
    dic_productos['Nombre'].append(nombre[0].text.strip())
    dic_productos['Descripción'].append(descripcion[0].text.strip())


pd.DataFrame(dic_productos)



,Nombre,Descripción
0,SanFrutos Vikingathor Boris Brew,"Cerveza de 8.2% ABV intensa, oscura y muy lupu..."
1,"Boon Oude Kriek 37,5cl",Cerveza de fermentación espontánea (Lambic) de...
2,Störtebeker Schwarz-Bier,Cerveza negra de baja fermentación de estilo S...
3,Orval,Orval es una cerveza tipo ale y es la que prin...
4,Augustiner Lagerbier Hell,Cerveza de baja fermentación de estilo Munich ...
...,...,...
583,Samuel Adams Boston Lager,Apariencia: color ambarino cobrizo y espuma pe...
584,Westmalle Dubbel,Cerveza Trapista belga con 7% ABV de color mar...
585,Duchesse De Bourgogne,Cerveza belga de estilo Ale Roja de 6.2% ABV d...
586,Tripel Karmeliet 33 cl.,Cerveza de estilo Belgian Tripel con 8.4% ABV....


### Ya sabemos obtener todos los datos que nos interesan de una cerveza, ahora tenemos que aplicar esta lógica para obtener todas las demás 

Construir un DataFrame con 

Nombre,	precio,	descripcion corta, descripción larga, ruta imagen, marca, barcode,	Estilo, Origen, % Alc., % Alc. Exacto, Volumen (Cl), Tipo Fermentación, IBU, IBU Exacto, Color, Envase

In [82]:
import requests
from bs4 import BeautifulSoup


data_beers = {"Nombre":[], "Descripcion":[], "Img":[], "Estilo":[]}

for producto in productos:
    response = requests.get(producto)
    soup = BeautifulSoup(response.content, 'html.parser')
    try:
        nombre = soup.find_all('h1', {'class': 'product-detail-name'})
    except:
        nombre = ""
    try:
        descripcion = soup.find_all('div', {'class': 'product-description'})
    except:
        descripcion = ""
    try:
        img = soup.find('img', {'class': 'js-qv-product-cover img-fluid'})
    except:
        img = ""
    try:
        estilo = soup.find('dd', {'class': 'value'})
    except:
        estilo = ""

    data_beers['Nombre'].append(nombre[0].text.strip() if nombre else "")
    data_beers['Descripcion'].append(descripcion[0].text.strip() if descripcion else "")
    data_beers['Img'].append(img["src"] if img else "")
    data_beers['Estilo'].append(estilo.text.strip() if estilo else "")

In [83]:
pd.DataFrame(data_beers)

,Nombre,Descripcion,Img,Estilo
0,SanFrutos Vikingathor Boris Brew,"Cerveza de 8.2% ABV intensa, oscura y muy lupu...",https://www.labirratorium.com/27941-large_defa...,DUNKEL
1,"Boon Oude Kriek 37,5cl",Cerveza de fermentación espontánea (Lambic) de...,https://www.labirratorium.com/19351-large_defa...,KRIEK
2,Störtebeker Schwarz-Bier,Cerveza negra de baja fermentación de estilo S...,https://www.labirratorium.com/488-large_defaul...,SCHWARZBIER
3,Orval,Orval es una cerveza tipo ale y es la que prin...,https://www.labirratorium.com/385-large_defaul...,BELGIAN PALE ALE
4,Augustiner Lagerbier Hell,Cerveza de baja fermentación de estilo Munich ...,https://www.labirratorium.com/367-large_defaul...,MUNICH HELLES
...,...,...,...,...
583,Samuel Adams Boston Lager,Apariencia: color ambarino cobrizo y espuma pe...,https://www.labirratorium.com/343-large_defaul...,AMERICAN LAGER
584,Westmalle Dubbel,Cerveza Trapista belga con 7% ABV de color mar...,https://www.labirratorium.com/102-large_defaul...,DUBBEL
585,Duchesse De Bourgogne,Cerveza belga de estilo Ale Roja de 6.2% ABV d...,https://www.labirratorium.com/12300-large_defa...,FLANDERS RED ALE\nOUD BRUIN\nOud Bruin / Frand...
586,Tripel Karmeliet 33 cl.,Cerveza de estilo Belgian Tripel con 8.4% ABV....,https://www.labirratorium.com/81-large_default...,Bélgica
